### First load dataset

In [107]:
import os
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [108]:
import tensorflow as tf

In [109]:
X_train = mnist.train.images
X_test = mnist.test.images
y_train = mnist.train.labels.astype("int")
y_test = mnist.test.labels.astype("int")
y_test

array([[0, 0, 0, ..., 1, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

# Now Start to construct model v1

comment: too slow

## Define some common attributes

In [131]:
n_epochs = 300
batch_size = 100

n_input = 28*28
n_layer1 = 400
n_layer2 = 200
n_output = 10

## The planning phase

In [132]:
tf.reset_default_graph()

In [133]:
X = tf.placeholder(tf.float64, [None, n_input], "X")
y = tf.placeholder(tf.float64, [None, n_output], "y")

In [134]:
with tf.name_scope("ann"):
    hidden1 = tf.layers.dense(X, n_layer1, name="hidden1", activation=tf.nn.relu)
    hidden2 = tf.layers.dense(hidden1, n_layer2, name="hidden2", activation=tf.nn.relu)
    output = tf.layers.dense(hidden2, n_output, name="output")

In [135]:
with tf.name_scope("loss"):
    xentropy = tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=output, name="xentropy")
    loss = tf.reduce_mean(xentropy, name="loss")

In [136]:
learning_rate = 0.01
with tf.name_scope("train"):
    adam_optimizer = tf.train.AdamOptimizer(learning_rate)
    training_op = adam_optimizer.minimize(loss)

In [137]:
with tf.name_scope("eval"):
    # a metrix with entries 1 if logits predict the same as class y
    arg_y = tf.argmax(y, axis=1)
    arg_y_pred = tf.argmax(output, axis=1)
    acc_all, acc = tf.metrics.accuracy(arg_y, arg_y_pred, name="accuracy")

In [138]:
from datetime import datetime

time_now = datetime.now().strftime("%Y-%m-%d=%H:%M:%S")

with tf.name_scope("log"):
    file_writer = tf.summary.FileWriter("logs/{}".format(time_now), tf.get_default_graph())
    acc_log = tf.summary.scalar("accuracy", acc)

In [139]:
saving_path = "temp/save/{}".format(time_now)

if not os.path.isdir(saving_path):
    os.makedirs(saving_path)

with tf.name_scope("saver"):
    saver = tf.train.Saver()

In [140]:
with tf.name_scope("initializer"):
    initializers = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())

In [141]:
with tf.Session() as sess:
    initializers.run()
    for epoch in range(n_epochs):
        for iteration in range(X_train.shape[0] // batch_size + 1):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
            
            
        if not (epoch%20):
    ##### !!!!!!! Dont evaluate acc each batch!!!! BUg!!!!!!!!!!!!!
            acc_train = acc.eval(feed_dict={X: X_batch, y: y_batch})
            acc_val, str_acc = sess.run([acc, acc_log], feed_dict={X: mnist.validation.images,
                                                                                 y: mnist.validation.labels})
            file_writer.add_summary(str_acc, epoch)
            saver.save(sess, "{}/{}.ckpt".format(saving_path, epoch//20))
            print(epoch, "Train acc:", acc_train, "Val acc:", acc_val)
            print(epoch, "loss=", loss.eval(feed_dict={X:X_batch, y:y_batch}))

    # Can use this with global to restore from the most recent run
    save_path = saver.save(sess, "{}/final.ckpt".format(saving_path))
    file_writer.close()

0 Train acc: 0.98 Val acc: 0.956471
0 loss= 0.0608635468541
20 Train acc: 0.957115 Val acc: 0.96549
20 loss= 0.0480413646043
40 Train acc: 0.965825 Val acc: 0.968562
40 loss= 0.000158997125522
60 Train acc: 0.968766 Val acc: 0.969608
60 loss= 0.00306137385281
80 Train acc: 0.969707 Val acc: 0.97051
80 loss= 0.0212749368892


KeyboardInterrupt: 

# Now Start to construct model v2

comment: bad

## Define some common attributes

In [120]:
n_epochs = 300
batch_size = 100

n_input = 28*28
n_layer1 = 300
n_layer2 = 100
n_layer3 = 33
n_layer4 = 15
n_output = 10

## The planning phase

In [121]:
tf.reset_default_graph()

In [122]:
X = tf.placeholder(tf.float64, [None, n_input], "X")
y = tf.placeholder(tf.float64, [None, n_output], "y")

In [123]:
with tf.name_scope("ann"):
    hidden1 = tf.layers.dense(X, n_layer1, name="hidden1", activation=tf.nn.relu)
    hidden2 = tf.layers.dense(hidden1, n_layer2, name="hidden2", activation=tf.nn.relu)
    hidden3 = tf.layers.dense(hidden2, n_layer3, name="hidden3", activation=tf.nn.relu)
    hidden4 = tf.layers.dense(hidden3, n_layer4, name="hidden4", activation=tf.nn.relu)
    output = tf.layers.dense(hidden4, n_output, name="output")

In [124]:
with tf.name_scope("loss"):
    xentropy = tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=output, name="xentropy")
    loss = tf.reduce_mean(xentropy, name="loss")

In [125]:
learning_rate = 0.01
with tf.name_scope("train"):
    adam_optimizer = tf.train.AdamOptimizer(learning_rate)
    training_op = adam_optimizer.minimize(loss)

In [126]:
with tf.name_scope("eval"):
    # a metrix with entries 1 if logits predict the same as class y
    arg_y = tf.argmax(y, axis=1)
    arg_y_pred = tf.argmax(output, axis=1)
    acc_all, acc = tf.metrics.accuracy(arg_y, arg_y_pred, name="accuracy")

In [127]:
from datetime import datetime

time_now = datetime.now().strftime("%Y-%m-%d=%H:%M:%S")

with tf.name_scope("log"):
    file_writer = tf.summary.FileWriter("logs/{}".format(time_now), tf.get_default_graph())
    acc_log = tf.summary.scalar("accuracy", acc)

In [128]:
saving_path = "temp/save/{}".format(time_now)

if not os.path.isdir(saving_path):
    os.makedirs(saving_path)

with tf.name_scope("saver"):
    saver = tf.train.Saver()

In [129]:
with tf.name_scope("initializer"):
    initializers = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())

In [130]:
with tf.Session() as sess:
    initializers.run()
    for epoch in range(n_epochs):
        for iteration in range(X_train.shape[0] // batch_size + 1):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
            
            
        if not (epoch%20):
    ##### !!!!!!! Dont evaluate acc each batch!!!! BUg!!!!!!!!!!!!!
            acc_train = acc.eval(feed_dict={X: X_batch, y: y_batch})
            acc_val, str_acc = sess.run([acc, acc_log], feed_dict={X: mnist.validation.images,
                                                                                 y: mnist.validation.labels})
            file_writer.add_summary(str_acc, epoch)
            recent_path = "{}/{}.ckpt".format(saving_path, epoch//20)
            global recent_path
            saver.save(sess, recent_path)
            print(epoch, "Train acc:", acc_train, "Val acc:", acc_val)
            print(epoch, "loss=", loss.eval(feed_dict={X:X_batch, y:y_batch}))

    # Can use this with global to restore from the most recent run
    save_path = saver.save(sess, "{}/final.ckpt".format(saving_path))
    file_writer.close()

0 Train acc: 0.96 Val acc: 0.960588
0 loss= 0.106491546
20 Train acc: 0.961346 Val acc: 0.968333
20 loss= 0.000938012674273
40 Train acc: 0.968641 Val acc: 0.969739
40 loss= 0.000336676375869
60 Train acc: 0.969935 Val acc: 0.970441
60 loss= 0.00376661113818
80 Train acc: 0.970488 Val acc: 0.970078
80 loss= 0.0501652779507
100 Train acc: 0.970039 Val acc: 0.968235
100 loss= 0.163446326232
120 Train acc: 0.967948 Val acc: 0.951905
120 loss= 0.34471344596
140 Train acc: 0.950754 Val acc: 0.906985
140 loss= 0.904730425202
160 Train acc: 0.905501 Val acc: 0.839368
160 loss= 1.46132619583
180 Train acc: 0.837891 Val acc: 0.776412
180 loss= 1.83425543922


KeyboardInterrupt: 

FUcking overfitting lahsdlfakn dkf1asfd @#$ Q@$T

# Now Start to construct model v3

comment: use 2 layers

## Define some common attributes

In [145]:
n_epochs = 300
batch_size = 100

n_input = 28*28
n_layer1 = 300
n_layer2 = 100
n_output = 10

## The planning phase

In [146]:
tf.reset_default_graph()

In [147]:
X = tf.placeholder(tf.float64, [None, n_input], "X")
y = tf.placeholder(tf.float64, [None, n_output], "y")

In [148]:
with tf.name_scope("ann"):
    hidden1 = tf.layers.dense(X, n_layer1, name="hidden1", activation=tf.nn.relu)
    hidden2 = tf.layers.dense(hidden1, n_layer2, name="hidden2", activation=tf.nn.relu)
    output = tf.layers.dense(hidden2, n_output, name="output")

In [149]:
with tf.name_scope("loss"):
    xentropy = tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=output, name="xentropy")
    loss = tf.reduce_mean(xentropy, name="loss")

In [150]:
learning_rate = 0.01
with tf.name_scope("train"):
    adam_optimizer = tf.train.AdamOptimizer(learning_rate)
    training_op = adam_optimizer.minimize(loss)

In [151]:
with tf.name_scope("eval"):
    # a metrix with entries 1 if logits predict the same as class y
    arg_y = tf.argmax(y, axis=1)
    arg_y_pred = tf.argmax(output, axis=1)
    acc_all, acc = tf.metrics.accuracy(arg_y, arg_y_pred, name="accuracy")

In [152]:
from datetime import datetime

time_now = datetime.now().strftime("%Y-%m-%d=%H:%M:%S")

with tf.name_scope("log"):
    file_writer = tf.summary.FileWriter("logs/{}".format(time_now), tf.get_default_graph())
    acc_log = tf.summary.scalar("accuracy", acc)

In [153]:
saving_path = "temp/save/{}".format(time_now)

if not os.path.isdir(saving_path):
    os.makedirs(saving_path)

with tf.name_scope("saver"):
    saver = tf.train.Saver()

In [154]:
with tf.name_scope("initializer"):
    initializers = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())

In [156]:
with tf.Session() as sess:
    initializers.run()
    for epoch in range(n_epochs):
        for iteration in range(X_train.shape[0] // batch_size + 1):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
            
            
        if not (epoch%20):
    ##### !!!!!!! Dont evaluate acc each batch!!!! BUg!!!!!!!!!!!!!
            acc_train = acc.eval(feed_dict={X: X_batch, y: y_batch})
            acc_val, str_acc = sess.run([acc, acc_log], feed_dict={X: mnist.validation.images,
                                                                     y: mnist.validation.labels})
            file_writer.add_summary(str_acc, epoch)
            save_path = saver.save(sess, "{}/{}.ckpt".format(saving_path, epoch//20))
            print(epoch, "Train acc:", acc_train, "Val acc:", acc_val)
            print(epoch, "loss=", loss.eval(feed_dict={X:X_batch, y:y_batch}))

    # Can use this with global to restore from the most recent run
    save_path = saver.save(sess, "{}/final.ckpt".format(saving_path))
    file_writer.close()

0 Train acc: 0.95 Val acc: 0.959412
0 loss= 0.166441789127
20 Train acc: 0.960192 Val acc: 0.967451
20 loss= 0.0101078848715
40 Train acc: 0.967767 Val acc: 0.968889
40 loss= 0.00213528188394
60 Train acc: 0.969091 Val acc: 0.969804
60 loss= 6.04461299629e-05
80 Train acc: 0.969805 Val acc: 0.969922
80 loss= 0.0876250391634
100 Train acc: 0.970039 Val acc: 0.971405
100 loss= 4.520523842e-08
120 Train acc: 0.971466 Val acc: 0.971569
120 loss= 0.0265208022615


KeyboardInterrupt: 

## Adding a test section

In [158]:
with tf.Session() as sess:
    initializers.run()
    saver.restore(sess, save_path)
    acc_test = acc.eval(feed_dict={X:X_test, y:y_test})
    print("The accuracy for test set is: {:<.4f}".format(acc_test))

INFO:tensorflow:Restoring parameters from temp/save/2018-01-16=00:56:18/6.ckpt
The accuracy for test set is: 0.9715


# Now Start to construct model v4

comment: Adam is bad, change to normal GD

## Define some common attributes

In [161]:
n_epochs = 300
batch_size = 100

n_input = 28*28
n_layer1 = 300
n_layer2 = 100
n_output = 10

## The planning phase

In [183]:
tf.reset_default_graph()

In [184]:
X = tf.placeholder(tf.float64, [None, n_input], "X")
y = tf.placeholder(tf.float64, [None, n_output], "y")

In [185]:
with tf.name_scope("ann"):
    hidden1 = tf.layers.dense(X, n_layer1, name="hidden1", activation=tf.nn.relu)
    hidden2 = tf.layers.dense(hidden1, n_layer2, name="hidden2", activation=tf.nn.relu)
    output = tf.layers.dense(hidden2, n_output, name="output")

In [186]:
with tf.name_scope("loss"):
    xentropy = tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=output, name="xentropy")
    loss = tf.reduce_mean(xentropy, name="loss")

In [187]:
learning_rate = 0.05
with tf.name_scope("train"):
    gradient_optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = gradient_optimizer.minimize(loss)

In [188]:
with tf.name_scope("eval"):
    # a metrix with entries 1 if logits predict the same as class y
    arg_y = tf.argmax(y, axis=1)
    arg_y_pred = tf.argmax(output, axis=1)
    acc_all, acc = tf.metrics.accuracy(arg_y, arg_y_pred, name="accuracy")

In [189]:
from datetime import datetime

time_now = datetime.now().strftime("%Y-%m-%d=%H:%M:%S")

with tf.name_scope("log"):
    file_writer = tf.summary.FileWriter("logs/{}".format(time_now), tf.get_default_graph())
    acc_log = tf.summary.scalar("accuracy", acc)

In [190]:
saving_path = "temp/save/{}".format(time_now)

if not os.path.isdir(saving_path):
    os.makedirs(saving_path)

with tf.name_scope("saver"):
    saver = tf.train.Saver()

In [191]:
with tf.name_scope("initializer"):
    initializers = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())

In [193]:
max_rep = 8
max_val_acc = 0
best_save_path = None

with tf.Session() as sess:
    initializers.run()
    rep = 0
    for epoch in range(n_epochs):
        for iteration in range(X_train.shape[0] // batch_size + 1):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

            
        if not (epoch%5):
    ##### !!!!!!! Dont evaluate acc each batch!!!! BUg!!!!!!!!!!!!!
            acc_train = acc.eval(feed_dict={X: X_batch, y: y_batch})
            acc_val, str_acc = sess.run([acc, acc_log], feed_dict={X: mnist.validation.images,
                                                                     y: mnist.validation.labels})

                
            file_writer.add_summary(str_acc, epoch)
            save_path = saver.save(sess, "{}/{}.ckpt".format(saving_path, epoch//20))
            
            
            if acc_val > max_val_acc:
                best_save_path = save_path
                min_val_acc = acc_val
                rep = 0
            else:
                print("incrementing rep")
                rep += 1
            
            print(epoch, "Train acc:", acc_train, "Val acc:", acc_val)
            print(epoch, "loss=", loss.eval(feed_dict={X:X_batch, y:y_batch}))
            
            if rep > max_rep:
                print("Overfitting: early terminating")
                break

    # Can use this with global to restore from the most recent run
    save_path = saver.save(sess, "{}/final.ckpt".format(saving_path))
    file_writer.close()

0 Train acc: 0.93 Val acc: 0.92902
0 loss= 0.268428471313
5 Train acc: 0.930192 Val acc: 0.948529
5 loss= 0.103769925192
10 Train acc: 0.949029 Val acc: 0.95732
10 loss= 0.0343264129421
15 Train acc: 0.957597 Val acc: 0.962598
15 loss= 0.0264392795545
20 Train acc: 0.96278 Val acc: 0.966471
20 loss= 0.00751361838445
25 Train acc: 0.966602 Val acc: 0.969118
25 loss= 0.0036491574307
30 Train acc: 0.969218 Val acc: 0.97112
30 loss= 0.00539456683715
35 Train acc: 0.971201 Val acc: 0.972623
35 loss= 0.00471257621901
40 Train acc: 0.97269 Val acc: 0.973813
40 loss= 0.00275044985103
45 Train acc: 0.97387 Val acc: 0.974765
45 loss= 0.00213583883691
50 Train acc: 0.974814 Val acc: 0.975615
50 loss= 0.00129352378819
55 Train acc: 0.975658 Val acc: 0.976274
55 loss= 0.00221665690016
60 Train acc: 0.976313 Val acc: 0.976863
60 loss= 0.00254346590089
65 Train acc: 0.976898 Val acc: 0.977339
65 loss= 0.000419140842244
70 Train acc: 0.977371 Val acc: 0.977712
70 loss= 0.00058465872688
75 Train acc: 0

## Adding a test section

In [194]:
with tf.Session() as sess:
    initializers.run()
    if best_save_path:
        saver.restore(sess, best_save_path)
    else:
        saver.restore(sess, save_path)
    acc_test = acc.eval(feed_dict={X:X_test, y:y_test})
    print("The accuracy for test set is: {:<.4f}".format(acc_test))

INFO:tensorflow:Restoring parameters from temp/save/2018-01-16=01:52:30/14.ckpt
The accuracy for test set is: 0.9812


Alright, the accuracy is pretty good this time around